<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1LeagueScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen
from contextlib import closing
from operator import attrgetter
from pprint import pprint
from subprocess import DEVNULL
from subprocess import Popen
import time


# Team Dic

In [3]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmpDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126','名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

tmp2Dic = {'清水':'126', '福岡':'135'}

J12021TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '徳島':'toku', '福岡':'fuku', '鳥栖':'tosu', '大分':'oita'}

J12020TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '鳥栖':'tosu', '大分':'oita'}

J12020TeamDic = {'札幌':'sapp', '仙台':'send', '鹿島':'kasm', '浦和':'uraw', '柏':'kasw', 'FC東京':'fctk', '川崎Ｆ':'ka-f', '横浜FM':'y-fm', '横浜FC':'y-fc' , '湘南':'shon', '清水':'shim', '名古屋':'nago', 'Ｇ大阪':'g-os','Ｃ大阪':'c-os', '神戸':'kobe','広島':'hiro', '鳥栖':'tosu', '大分':'oita'}


def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

get_keys_from_value(TeamDic,'276')

['札幌']

<select id="comparison1_team"><option value="0">チーム選択</option><option value="276">札幌</option><option value="31219">八戸</option><option value="30673">岩手</option><option value="238">仙台</option><option value="30310">秋田</option><option value="294">山形</option><option value="30674">福島</option><option value="120">鹿島</option><option value="176">水戸</option><option value="30305">栃木</option><option value="30103">群馬</option><option value="122">浦和</option><option value="199">大宮</option><option value="121">千葉</option><option value="132">柏</option><option value="270">FC東京</option><option value="123">東京Ｖ</option><option value="30532">町田</option><option value="86">川崎Ｆ</option><option value="124">横浜FM</option><option value="296">横浜FC</option><option value="30675">YS横浜</option><option value="130">湘南</option><option value="30676">相模原</option><option value="150">甲府</option><option value="30528">松本</option><option value="30535">長野</option><option value="193">新潟</option><option value="30308">富山</option><option value="30526">金沢</option><option value="126">清水</option><option value="131">磐田</option><option value="30677">藤枝</option><option value="31296">沼津</option><option value="127">名古屋</option><option value="30302">岐阜</option><option value="134">京都</option><option value="128">Ｇ大阪</option><option value="133">Ｃ大阪</option><option value="136">神戸</option><option value="30000">鳥取</option><option value="30314">岡山</option><option value="129">広島</option><option value="30851">山口</option><option value="30536">讃岐</option><option value="30116">徳島</option><option value="30148">愛媛</option><option value="30974">今治</option><option value="135">福岡</option><option value="30313">北九州</option><option value="269">鳥栖</option><option value="30531">長崎</option><option value="30303">熊本</option><option value="207">大分</option><option value="31508">宮崎</option><option value="31042">鹿児島</option><option value="30317">琉球</option></select>

# ScrapeData

In [9]:
def scrapeData(homeTeamID,awayTeamID,year):
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)

    #試合結果を取得
    html = requests.get(url)

    html.encoding = "UTF-8"
    soup = BeautifulSoup(html.text, "html.parser")

    texts = soup.find('div', attrs={'class': 'gameTbl'}).text
    texts = re.findall(r'\w+', texts)
    time.sleep(1) 

    return df,texts

# Scrape Season Data

In [13]:
def scrapeSeasonData(TeamNameDic,dfList,year):
    for team in TeamNameDic:
        url = 'https://www.football-lab.jp/' +  TeamNameDic[team] + '/match/' + '?year=' + year
        #表を取得
        df = pd.read_html(url)[0]
        #最後の列を削除
        if (year == '2021'):
            df = df.drop(38).drop(df.columns[[2,6,7,8,19,20]], axis=1)
        else:
            df = df.drop(34).drop(df.columns[[2,6,7,8,19,20]], axis=1)
        #homeTeam名追加
        df.insert(2,'Team',team)
        #列名を英語に
        df = df.rename(columns={'相手': 'awayTeam','Unnamed: 5':'H or A','チャンス構築率':'CBP','シュート':'shot','シュート成功率':'shot acc','支配率':'possession','攻撃CBP':'ACBP','パスCBP':'PassCBP','奪取P':'InterceptP','守備P':'DP'})
        #Home Awayのチーム並びかえ
        df['Team'].where(df['H or A']=='H', df['awayTeam'] ,inplace = True)
        df['awayTeam'].where(df['H or A']=='H', team ,inplace = True)
        #Home Awayのスコアに修正
        df.insert(4,'awayScore',0)
        df.insert(4,'homeScore',0)
        #Teamがhomeの時のscore入力
        df['homeScore'].where(df['H or A']=='A',df['スコア'].str[:1],inplace = True)
        df['awayScore'].where(df['H or A']=='A',df['スコア'].str[2:3],inplace = True)
        #Teamがawayの時のscore入力
        df['awayScore'].where(df['H or A']=='H',df['スコア'].str[:1],inplace = True)
        df['homeScore'].where(df['H or A']=='H',df['スコア'].str[2:3],inplace = True)
        df = df.drop(df.columns[[6]], axis=1)
        dfList.append(df)
        time.sleep(1) 
    return dfList

In [14]:
tmpJ12020TeamDic = {'札幌':'sapp', '仙台':'send',}
dfList = []
dfList =  scrapeSeasonData(tmpJ12020TeamDic,dfList,'2020')
dfList[0]

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A,AGI,KAGI,CBP,shot,shot acc,possession,ACBP,PassCBP,InterceptP,DP
0,1,2.22,柏,札幌,4,2,A,61,44,17.3%,27,7.4%,50.6%,24.59,15.50,106.59,13.83
1,2,7.4,横浜FC,札幌,1,2,A,48,41,11.1%,12,16.7%,42.7%,15.91,12.13,71.45,15.38
2,3,7.8,鹿島,札幌,0,2,A,34,53,6.5%,7,28.6%,46.2%,13.19,11.99,76.32,17.79
3,4,7.12,湘南,札幌,0,0,A,50,50,6.7%,7,0.0%,47.4%,16.32,13.93,70.91,14.59
4,5,7.18,仙台,札幌,2,2,A,49,45,13.6%,15,13.3%,50.4%,15.33,12.20,61.17,14.91
5,6,7.22,札幌,FC東京,1,1,H,45,55,8.6%,10,10.0%,48.0%,15.11,10.78,105.26,12.52
6,7,7.26,札幌,横浜FM,3,1,H,48,52,11.8%,15,20.0%,38.1%,11.90,9.46,124.57,13.42
7,8,8.2,札幌,神戸,2,3,H,56,62,15.8%,16,12.5%,50.2%,21.51,15.85,74.22,8.92
8,9,8.8,清水,札幌,3,1,A,49,42,13.0%,14,7.1%,50.6%,16.56,13.37,67.38,13.15
9,10,8.15,札幌,川崎Ｆ,1,6,H,57,50,10.6%,12,8.3%,48.0%,18.21,13.84,101.54,9.44


In [16]:
SeasonData = []
SeasonData =  scrapeSeasonData(J12020TeamDic,SeasonData,'2020')
SeasonData

[     節    開催日  Team awayTeam homeScore awayScore H or A AGI KAGI    CBP shot  \
 0    1   2.22     柏       札幌         4         2      A  61   44  17.3%   27   
 1    2    7.4  横浜FC       札幌         1         2      A  48   41  11.1%   12   
 2    3    7.8    鹿島       札幌         0         2      A  34   53   6.5%    7   
 3    4   7.12    湘南       札幌         0         0      A  50   50   6.7%    7   
 4    5   7.18    仙台       札幌         2         2      A  49   45  13.6%   15   
 5    6   7.22    札幌     FC東京         1         1      H  45   55   8.6%   10   
 6    7   7.26    札幌     横浜FM         3         1      H  48   52  11.8%   15   
 7    8    8.2    札幌       神戸         2         3      H  56   62  15.8%   16   
 8    9    8.8    清水       札幌         3         1      A  49   42  13.0%   14   
 9   10   8.15    札幌      川崎Ｆ         1         6      H  57   50  10.6%   12   
 10  11   8.19    札幌       大分         1         1      H  61   77  16.1%   20   
 11  29   8.26  横浜FM       札

In [17]:
tmp = SeasonData[0].merge(SeasonData[1], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
tmp2 = SeasonData[0].merge(SeasonData[2], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
tmp3 = tmp.merge(tmp2, how="outer")
tmp3

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
0,1,2.22,柏,札幌,4,2,A,61,44,17.3%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,7.4,横浜FC,札幌,1,2,A,48,41,11.1%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7.8,鹿島,札幌,0,2,A,34,53,6.5%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,7.12,湘南,札幌,0,0,A,50,50,6.7%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,7.18,仙台,札幌,2,2,A,49,45,13.6%,...,55,51,17.1%,19,10.5%,49.6%,17.16,11.86,103.35,12.69
5,6,7.22,札幌,FC東京,1,1,H,45,55,8.6%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,7.26,札幌,横浜FM,3,1,H,48,52,11.8%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,8.2,札幌,神戸,2,3,H,56,62,15.8%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,8.8,清水,札幌,3,1,A,49,42,13.0%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,8.15,札幌,川崎Ｆ,1,6,H,57,50,10.6%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
seasonDataList = []
for i in range (len(SeasonData)):
    if(i!=0):
        tmp0 = SeasonData[i].merge(SeasonData[0], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=1):
        tmp1 = SeasonData[i].merge(SeasonData[1], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=2):
        tmp2 = SeasonData[i].merge(SeasonData[2], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=3):
        tmp3 = SeasonData[i].merge(SeasonData[3], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=4):
        tmp4 = SeasonData[i].merge(SeasonData[4], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=5):
        tmp5 = SeasonData[i].merge(SeasonData[5], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=6):
        tmp6 = SeasonData[i].merge(SeasonData[6], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=7):
        tmp7 = SeasonData[i].merge(SeasonData[7], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=8):
        tmp8 = SeasonData[i].merge(SeasonData[8], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=9):
        tmp9 = SeasonData[i].merge(SeasonData[9], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=10):
        tmp10 = SeasonData[i].merge(SeasonData[10], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=11):
        tmp11 = SeasonData[i].merge(SeasonData[11], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=12):
        tmp12 = SeasonData[i].merge(SeasonData[12], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=13):
        tmp13 = SeasonData[i].merge(SeasonData[13], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=14):
        tmp14 = SeasonData[i].merge(SeasonData[14], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=15):
        tmp15 = SeasonData[i].merge(SeasonData[15], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=16):
        tmp16 = SeasonData[i].merge(SeasonData[16], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=17):
        tmp17= SeasonData[i].merge(SeasonData[17], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(len(SeasonData) ==19):
        if(i!=18):
            tmp18 = SeasonData[i].merge(SeasonData[18], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
        if(i!=19):
            tmp19 = SeasonData[i].merge(SeasonData[19], how="left",on=['節', '開催日','Team','awayTeam','homeScore','awayScore'])
    if(i!=1 and i!=2):
        tmp = tmp1.merge(tmp2, how="outer")
    if(i!=0 and i!=2):
        tmp = tmp0.merge(tmp2, how="outer")
    if(i!=0 and i!=1):
        tmp = tmp0.merge(tmp1, how="outer")
    if(i!=0 and i!=1 and i!=2):
        tmp = tmp.merge(tmp2, how="outer")
    if(i!=3):
        tmp = tmp.merge(tmp3, how="outer")
    if(i!=4):
        tmp = tmp.merge(tmp4, how="outer")
    if(i!=5):
        tmp = tmp.merge(tmp5, how="outer")
    if(i!=6):
        tmp = tmp.merge(tmp6, how="outer")
    if(i!=7):
        tmp = tmp.merge(tmp7, how="outer")
    if(i!=8):
        tmp = tmp.merge(tmp8, how="outer")
    if(i!=9):
        tmp = tmp.merge(tmp9, how="outer")
    if(i!=10):
        tmp = tmp.merge(tmp10, how="outer")
    if(i!=11):
        tmp = tmp.merge(tmp11, how="outer")
    if(i!=12):
        tmp = tmp.merge(tmp12, how="outer")
    if(i!=13):
        tmp = tmp.merge(tmp13, how="outer")
    if(i!=14):
        tmp = tmp.merge(tmp14, how="outer")
    if(i!=15):
        tmp = tmp.merge(tmp15, how="outer")
    if(i!=16):
        tmp = tmp.merge(tmp16, how="outer")
    if(i!=17):
        tmp = tmp.merge(tmp17, how="outer")
    if(len(SeasonData) ==19):
        if(i!=18):
            tmp = tmp.merge(tmp18, how="outer")
        if(i!=19):
            tmp = tmp.merge(tmp19, how="outer")
    tmp = tmp.dropna(how='any')
    if(i!=0):
        seasonData = pd.concat([seasonData,tmp])
    else:
        seasonData = tmp
#.merge(tmp6, how="outer").merge(tmp7, how="outer").merge(tmp8, how="outer").merge(tmp9, how="outer").merge(tmp10, how="outer").merge(tmp11, how="outer").merge(tmp12, how="outer").merge(tmp13, how="outer").merge(tmp14, how="outer").merge(tmp15, how="outer").merge(tmp16, how="outer").merge(tmp17, how="outer").merge(tmp18, how="outer").merge(tmp19, how="outer")
seasonData

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
4,5,7.18,仙台,札幌,2,2,A,49,45,13.6%,...,55,51,17.1%,19,10.5%,49.6%,17.16,11.86,103.35,12.69
20,20,10.3,札幌,仙台,3,3,H,60,51,15.4%,...,49,40,12.1%,13,23.1%,36.1%,8.94,6.30,68.97,17.41
34,3,7.8,鹿島,札幌,0,2,A,34,53,6.5%,...,47,66,18.3%,20,0.0%,53.8%,16.57,15.09,108.15,11.59
37,23,10.18,札幌,鹿島,1,0,H,46,47,8.8%,...,53,54,6.3%,10,0.0%,53.8%,18.22,13.54,122.71,13.07
38,16,9.13,札幌,浦和,3,4,H,71,53,22.8%,...,47,29,7.8%,8,50.0%,38.9%,11.57,8.92,60.98,26.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,22,10.14,神戸,大分,1,1,A,44,62,10.8%,...,38,56,7.1%,8,12.5%,51.2%,15.02,11.68,86.08,13.78
64,3,7.8,広島,大分,1,2,A,40,47,15.8%,...,53,60,12.0%,12,8.3%,44.9%,17.37,13.44,56.77,13.14
65,18,9.23,大分,広島,0,2,H,34,70,6.8%,...,30,66,5.5%,5,40.0%,48.0%,11.50,10.87,74.37,9.47
66,2,7.4,大分,鳥栖,2,0,H,36,58,9.8%,...,42,64,13.0%,13,0.0%,50.0%,13.38,11.00,60.85,10.68


In [21]:
seasonData.to_csv("2020SeasonData.csv",encoding = 'utf-8-sig')

In [22]:
csv ='/content/2020SeasonData.csv'
data = pd.read_csv(csv)

In [23]:
#index列を削除
data = data.drop(data.columns[[0]], axis=1)
data

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
0,5,7.18,仙台,札幌,2,2,A,49,45,13.6%,...,55,51,17.1%,19,10.5%,49.6%,17.16,11.86,103.35,12.69
1,20,10.30,札幌,仙台,3,3,H,60,51,15.4%,...,49,40,12.1%,13,23.1%,36.1%,8.94,6.30,68.97,17.41
2,3,7.80,鹿島,札幌,0,2,A,34,53,6.5%,...,47,66,18.3%,20,0.0%,53.8%,16.57,15.09,108.15,11.59
3,23,10.18,札幌,鹿島,1,0,H,46,47,8.8%,...,53,54,6.3%,10,0.0%,53.8%,18.22,13.54,122.71,13.07
4,16,9.13,札幌,浦和,3,4,H,71,53,22.8%,...,47,29,7.8%,8,50.0%,38.9%,11.57,8.92,60.98,26.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,22,10.14,神戸,大分,1,1,A,44,62,10.8%,...,38,56,7.1%,8,12.5%,51.2%,15.02,11.68,86.08,13.78
608,3,7.80,広島,大分,1,2,A,40,47,15.8%,...,53,60,12.0%,12,8.3%,44.9%,17.37,13.44,56.77,13.14
609,18,9.23,大分,広島,0,2,H,34,70,6.8%,...,30,66,5.5%,5,40.0%,48.0%,11.50,10.87,74.37,9.47
610,2,7.40,大分,鳥栖,2,0,H,36,58,9.8%,...,42,64,13.0%,13,0.0%,50.0%,13.38,11.00,60.85,10.68


In [24]:
# Hが上に来る様に並び替え
data = data.sort_values('H or A_x', ascending=False)

In [25]:
#重複削除
data = data.drop_duplicates(subset=['節','開催日','Team','awayTeam'])

In [26]:
#節を昇順に並び替え
data = data.sort_values('節')
data

,節,開催日,Team,awayTeam,homeScore,awayScore,H or A_x,AGI_x,KAGI_x,CBP_x,...,AGI_y,KAGI_y,CBP_y,shot_y,shot acc_y,possession_y,ACBP_y,PassCBP_y,InterceptP_y,DP_y
260,1,2.23,横浜FM,Ｇ大阪,1,2,H,61,50,12.9%,...,50,39,7.3%,10,20.0%,34.4%,10.92,6.24,130.11,25.07
474,1,2.22,Ｃ大阪,大分,1,0,H,42,48,11.1%,...,52,58,18.4%,18,0.0%,54.8%,17.71,13.01,84.66,10.36
514,1,2.23,広島,鹿島,3,0,H,49,38,9.8%,...,62,51,15.6%,19,0.0%,63.2%,23.88,16.68,77.57,10.77
54,1,2.22,仙台,名古屋,1,1,H,50,44,9.2%,...,56,50,14.3%,20,5.0%,61.6%,17.55,14.06,108.09,13.13
492,1,2.23,神戸,横浜FC,1,1,H,61,69,14.2%,...,31,39,3.7%,4,25.0%,34.6%,7.42,5.99,76.12,22.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,34,12.19,柏,川崎Ｆ,2,3,H,44,44,11.2%,...,56,56,12.7%,17,17.6%,66.8%,27.53,21.55,116.50,14.39
287,34,12.19,横浜FC,横浜FM,3,1,H,45,31,11.3%,...,69,55,23.7%,27,3.7%,60.5%,26.16,21.74,92.40,8.95
429,34,12.19,Ｇ大阪,清水,0,2,H,59,53,14.2%,...,47,41,10.5%,12,16.7%,39.6%,10.43,9.13,104.66,22.77
103,34,12.19,浦和,札幌,0,2,H,38,36,7.9%,...,64,62,18.5%,20,10.0%,52.7%,24.38,18.41,115.62,11.12


In [27]:
#書き出し
data.to_csv("seasonData_2020.csv",encoding = 'utf-8-sig')